In [1]:
import pandas as pd
import numpy as np
import h5py
import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.nn.functional as func
from PIL import Image
from datasetCreator import ImageSubset
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from models import VGG

In [2]:
test = pd.read_csv('data/BNPP_DT_test_with_ages.csv')
val = pd.read_csv('data/BNPP_DT_val_with_ages.csv')
train = pd.read_csv('data/BNPP_DT_train_with_ages.csv')
f = h5py.File('data/bnpp_frontalonly_1024_10.hdf5')

In [3]:
train.iloc[6,:]

unique_key           Nisrana_50554206_img1
bnpp_value_log                    0.690196
BNPP_weight                              1
PNA_mask                              None
PNA_wight_mask                           1
BNP_value                              4.9
age_at_sampletime                       49
Name: 6, dtype: object

In [4]:
#list(f.keys())
data = {}
ind=0
for key in list(f.keys()):
    image = f[key][:]
    #resizing to 224 x 224
    formatted = (image * 255 / np.max(image)).astype('uint8')
    img = Image.fromarray(formatted).resize((224,224))
    data[ind]=[key,img]
    #img_tensor = transform(img)
    #data[key] = img_tensor
    img.save(f'data/images/{key}.jpg')
    ind+=1

In [5]:
transform = transforms.Compose([
    transforms.PILToTensor(),
    transforms.Normalize(mean=(0.5), std=(0.5))
])
train_data = ImageSubset(csv_file = 'data/BNPP_DT_train_with_ages.csv', data=data, img_dir='data/images/')
train_data

In [7]:
train[train['unique_key']=='Umellum_52452344_img1']
#train_data.__getitem__(6)

,unique_key,bnpp_value_log,BNPP_weight,PNA_mask,PNA_wight_mask,BNP_value,age_at_sampletime


In [ ]:
train_dataloader = DataLoader(train_data,batch_size=16)
train_dataloader

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = VGG('VGG16').to(device)
model

In [ ]:
x = torch.randn(2, 1, 224, 224)

In [ ]:
out = model(x)
out

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"[INFO]: Computation device: {device}")
epochs = 10
batch_size = 32

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
print(f"[INFO]: {total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"[INFO]: {total_trainable_params:,} trainable parameters.")
# the loss function
criterion = nn.L1Loss()
# the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.0005)

In [ ]:
# training
def train(model, trainloader, optimizer, criterion):
    model.train()
    print('Training')
    train_running_loss = 0.0
    train_running_correct = 0
    counter = 0
    for i, data in tqdm(enumerate(trainloader), total=len(trainloader)):
        counter += 1

        image, bnpp = data[0], data[2]
        #print(i)
        #print(data)
        #print(data[0])
        #print(data[1])
        #print(data[2])
        #break
        image = image.to(device)
        bnpp = bnpp.to(device)

        optimizer.zero_grad()
        # forward pass
        outputs = model(image.float())
        # calculate the loss
        loss = criterion(outputs, bnpp)
        train_running_loss += loss.item()
        # calculate the accuracy
        #_, preds = torch.max(outputs.data, 1)
        #train_running_correct += (preds == labels).sum().item()
        loss.backward()
        optimizer.step()

    epoch_loss = train_running_loss / counter
    #epoch_acc = 100. * (train_running_correct / len(trainloader.dataset))
    return epoch_loss, epoch_acc

In [ ]:
train(model, train_dataloader,optimizer,criterion)

In [ ]:
# our transforms will differ a bit from the VGG paper
# as we are using the MNIST dataset, so, we will directly resize...
# ... the images to 224x224 and not crop them and we will not use...
# ... any random flippings also
train_transform = transforms.Compose(
    [transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize(mean=(0.5), std=(0.5))])
valid_transform = transforms.Compose(
    [transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize(mean=(0.5), std=(0.5))])